<a href="https://colab.research.google.com/github/Walt9819/factores-impacto-desempleo-mexico/blob/main/Python/An%C3%A1lisisExploratorioDENUE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DENUE Y NOTAS PERIODISTICAS LA JORNADA**

Análisis exploratorio del conjunto de datos

https://www.inegi.org.mx/app/mapa/denue/default.aspx


In [8]:
import pandas as pd
import locale
from datetime import datetime
import plotly.express as px
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import dateutil

### Lectura de archivos

In [10]:
data = pd.read_csv('https://raw.githubusercontent.com/Walt9819/factores-impacto-desempleo-mexico/main/Python/newsscrapper/jornada_restaurantes.csv')
data.head(10)

,author,body,date,keyWords,link,location,source,title
0,Redacción,"México, como muchos países está volviendo a re...","martes, 29 dic 2020 09:00","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/economia/20...,NaN,LaJornada.com,2020: Un año donde la diferencia la hicimos to...
1,"Héctor Briseño, Corresponsal","Acapulco, Gro., Los principales destinos turís...","lunes, 04 ene 2021 07:51","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/estados/202...,NaN,LaJornada.com,Cayó $3 mil millones la derrama turística dece...
2,Afp,Londres. La totalidad de los 56 millones de ha...,"jueves, 26 nov 2020 14:40","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/mundo/2020/...,Londres,LaJornada.com,Inglaterra mantendrá importantes restricciones...
3,"Afp, Ap y Sputnik",Los Ángeles. En su lucha por controlar cifras ...,"jueves, 26 nov 2020 08:01","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/mundo/2020/...,NaN,LaJornada.com,Millón y medio de infectados en América en una...
4,Rocío González Alvarado,Sábado 28 de noviembre de 2020. Una de las apu...,"sábado, 28 nov 2020 08:24","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/capital/202...,NaN,LaJornada.com,"""Alerta al límite del rojo"" en la CDMX: Shein..."
5,Afp,Londres. Más de 60 personas fueron detenidas e...,"sábado, 28 nov 2020 15:33","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/mundo/2020/...,Londres.,LaJornada.com,Cientos protestan contra las restricciones Cov...
6,Josefina Quintero M.,Ciudad de México. La alerta al límite para pas...,"domingo, 29 nov 2020 08:16","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/capital/202...,Ciudad de México.,LaJornada.com,Nada frena las aglomeraciones en la CDMX
7,Alejandro Cruz Flores,Ciudad de México. El Gobierno de la Ciudad de ...,"lunes, 30 nov 2020 07:20","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/capital/202...,NaN,LaJornada.com,Sheinbaum refuerza las medidas contra aglomera...
8,Afp,"París. Europa, segunda región del mundo más en...","sábado, 28 nov 2020 15:11","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/mundo/2020/...,NaN,LaJornada.com,Supera Europa 400 mil muertos por Covid-19
9,Reuters,Tokio. Tokio elevó el jueves al nivel máximo s...,"jueves, 19 nov 2020 14:15","'restaurantes,covid'",https://www.jornada.com.mx/ultimas/mundo/2020/...,Tokio.,LaJornada.com,Eleva Tokio a nivel máximo su alerta sanitaria...


In [11]:
data['date'] = data['date'].str.replace(r'ago', 'aug')
data['date'] = data['date'].str.replace(r'abr', 'apr')
data['date'] = data['date'].str.replace(r'ene', 'jan')
data['date'] = data['date'].str.replace(r'dic', 'dec')
data['date'] = [dateutil.parser.parse(x, fuzzy_with_tokens=True)[0] for x in data['date']]
data['year'] = pd.DatetimeIndex(data['date']).year
data['month'] = pd.DatetimeIndex(data['date']).month

In [18]:
data

,author,body,date,keyWords,link,location,source,title,year,month
0,Redacción,"México, como muchos países está volviendo a re...",2020-12-29 09:00:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/economia/20...,NaN,LaJornada.com,2020: Un año donde la diferencia la hicimos to...,2020,12
1,"Héctor Briseño, Corresponsal","Acapulco, Gro., Los principales destinos turís...",2021-01-04 07:51:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/estados/202...,NaN,LaJornada.com,Cayó $3 mil millones la derrama turística dece...,2021,1
2,Afp,Londres. La totalidad de los 56 millones de ha...,2020-11-26 14:40:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/mundo/2020/...,Londres,LaJornada.com,Inglaterra mantendrá importantes restricciones...,2020,11
3,"Afp, Ap y Sputnik",Los Ángeles. En su lucha por controlar cifras ...,2020-11-26 08:01:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/mundo/2020/...,NaN,LaJornada.com,Millón y medio de infectados en América en una...,2020,11
4,Rocío González Alvarado,Sábado 28 de noviembre de 2020. Una de las apu...,2020-11-28 08:24:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/capital/202...,NaN,LaJornada.com,"""Alerta al límite del rojo"" en la CDMX: Shein...",2020,11
...,...,...,...,...,...,...,...,...,...,...
639,Néstor Jiménez y Alonso Urrutia,Ciudad de México. México es autosuficientes en...,2020-03-31 09:13:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/economia/20...,NaN,LaJornada.com,Multas de hasta 3 mdp por alzas irregulares en...,2020,3
640,Carlos Águila Arreola/ La Jornada Maya,"Cancún. Los cines, bares, discotecas, casinos ...",2020-03-18 20:03:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/estados/202...,Cancún,LaJornada.com,Cancún cerrará todo giro comercial para evitar...,2020,3
641,Lilián Anaya @Lilian_Anaya,Ciudad de México. Uber Eats informó que une es...,2020-03-23 16:00:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/tiempo-de-i...,Ciudad de México,LaJornada.com,"Uber Eats apoya a restaurantes, repartidores y...",2020,3
642,Ap,La Habana. Un millar de cruceristas y la tripu...,2020-03-18 15:40:00,"'restaurantes,covid'",https://www.jornada.com.mx/ultimas/mundo/2020/...,La Habana.,LaJornada.com,Desembarcan en Cuba pasajeros de crucero recha...,2020,3


In [20]:
count_year = data.groupby('year').count()[['link']].reset_index().rename(columns={'link': 'count'})
count_year

,year,count
0,2020,643
1,2021,1


In [16]:
count_month = data.groupby('month').count()[['link']].reset_index().rename(columns={'link': 'count'})
count_month.columns

Index(['month', 'count'], dtype='object')

In [17]:
fig = px.bar(count_month, x="month", y="count")
fig.show()

In [19]:
data_month = data.groupby(['month'])['body'].apply(' '.join).reset_index()
data_month = data_month.set_index('month')
data_month

,body
month,
1,"Acapulco, Gro., Los principales destinos turís..."
2,Seúl. Corea del Norte anunció un arsenal de me...
3,"Mérida, Yuc. Para disminuir en todo lo posible..."
4,"Teotihuacán, Méx. Un grupo de comerciantes y a..."
5,Berlín. Alemania prolongará hasta el 29 de jun...
6,Ciudad de México. De los 121 mil 765 estableci...
7,Lima. El gobierno peruano informó este miércol...
8,Londres. Las medidas para evitar la propagació...
9,Ciudad de México. Dirigentes de asociaciones d...


In [ ]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
 
round1 = lambda x: clean_text_round1(x)
 
data_clean = pd.DataFrame(data_year.body.apply(round1))
 
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text
 
round2 = lambda x: clean_text_round2(x)
 
data_clean = pd.DataFrame(data_clean.body.apply(round2))
data_clean
 
# Let's pickle it for later use
data_year.to_pickle("corpus.pkl")

### Filtrado por entidades